# Flipped Classroom 04: Training a Twin Model on MNIST 

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import datetime

%load_ext tensorboard

2022-11-19 21:42:40.717723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-19 21:42:41.211709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-19 21:42:41.211741: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-19 21:42:41.271826: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-19 21:42:42.774496: W tensorflow/stream_executor/platform/de

## Creating a dataset

**Task: take two random mnist digit 28x28x1 images and compute the equality of the numbers that they display.**


Input to the model will be: batch-size times two randomly chosen mnist digits, flattened to vectors.

Output of the model (its targets) should be: for each element in the batch, 1 if the two mnist images have the same label, and 0 otherwise.

*Notice how this will lead to an imbalanced classification dataset. We can address this by setting a sample_weight argument in the loss calculation, weighting the contribution to gradients stronger for the rare class than for the common class. We have similar issues with class imbalance in many applied fields (medicine, finance, fraud-detection etc.)*

In [3]:
# 1. get mnist from tensorflow_datasets
mnist = tfds.load("mnist", split =["train","test"], as_supervised=True)
train_ds = mnist[0]
val_ds = mnist[1]

# 2. write function to create the dataset that we want
def preprocess(data, batch_size):
    # image should be float
    data = data.map(lambda x, t: (tf.cast(x, float), t))
    # image should be flattened
    data = data.map(lambda x, t: (tf.reshape(x, (-1,)), t))
    # image vector will here have values between -1 and 1
    data = data.map(lambda x,t: ((x/128.)-1., t))
    # we want to have two mnist images in each example
    # this leads to a single example being ((x1,y1),(x2,y2))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), 
                                     data.shuffle(2000)))
    # map ((x1,y1),(x2,y2)) to (x1,x2, y1==y2*) *boolean
    zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], x1[1]==x2[1]))
    # transform boolean target to int
    zipped_ds = zipped_ds.map(lambda x1, x2, t: (x1,x2, tf.cast(t, tf.int32)))
    # batch the dataset
    zipped_ds = zipped_ds.batch(batch_size)
    # prefetch
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds

train_ds = preprocess(train_ds, batch_size=32) #train_ds.apply(preprocess)
val_ds = preprocess(val_ds, batch_size=32) #val_ds.apply(preprocess)

2022-11-19 21:42:46.824542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-19 21:42:46.824844: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-19 21:42:46.824870: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-ATOP1V9): /proc/driver/nvidia/version does not exist
2022-11-19 21:42:46.827363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# check the contents of the dataset
for img1, img2, label in train_ds.take(1):
    print(img1.shape, img2.shape, label.shape)

(32, 784) (32, 784) (32,)


2022-11-19 21:42:47.595328: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2022-11-19 21:42:47.597187: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Create a model for the task

- The two images should be processed using the same parameters (sometimes called a siamese or twin model)

- What do we need?

1. Subclass from **tf.keras.Model**

2. Implement **constructor method** that contains 
    - a) metrics objects to keep track (tf.keras.metrics)
    - b) optimizer
    - c) loss function
    - d) the layers that are used for prediction, incl. output layer
3. A **call method** that applies the computations (makes use of the model's layers)
    - has a training flag argument to allow for different training and inference behavior
    - uses the same layers for both images and then e.g. concatenates the representations before feeding them to an output layer.
    
4. A **metrics property** that returns a list of all metrics objects in the model

5. A **reset_metrics** method that iterates through the metrics in the metrics property and resets their state.
    - This is called before after each epoch and between training and validation during an epoch (to not mix train score with val score)
    
6. A **train_step** method that takes a data tuple as input, containing one batch of e.g. inputs, targets. In this case it is (inputs_1, inputs_2, target)
    - compute prediction and loss within a gradient tape context
    - get gradients w.r.t. trainable variables from the tape
    - pass the gradients and the trainable variables to the optimizer
    - update the metrics' states using the loss, the prediction and the target
    - return a dictionary containing each metric's name as a key and the corresponding result state as its value. This allows to easily print, and it is needed for the more high level compile and fit methods.
    
    
7. A **test_step** method that does the same as the train_step but without gradient calculations or optimization. 
    - The training flag in the model's call method should be set to False.

In [ ]:
class TwinMNISTModel(tf.keras.Model):

    # 1. constructor
    def __init__(self):
        super().__init__()
        # inherit functionality from parent class

        # optimizer, loss function and metrics
        self.metrics_list = [tf.keras.metrics.BinaryAccuracy(),
                             tf.keras.metrics.Mean(name="loss")]
        
        self.optimizer = tf.keras.optimizers.Adam()
        
        self.loss_function = tf.keras.losses.BinaryCrossentropy()
        
        # layers to be used
        self.dense1 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
        
        self.out_layer = tf.keras.layer.Dense(1,activation=tf.nn.sigmoid)
        
        
        
        
    # 2. call method (forward computation)
    def call(self, images, training=False):
        img1, img2 = images
        
        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)
        
        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)
        
        combined_x = tf.concat([img1_x, img2_x ], axis=1)
        
        return self.out_layer(combined_x)



    # 3. metrics property
    @property
    def metrics(self):
        return self.metrics_list
        # return a list with all metrics in the model



    # 4. reset all metrics objects
    def reset_metrics(self):
        for metric in self.metrics:
            metrics.reset_states()



    # 5. train step method
    def train_step(self, data):
        img1, img2, label = data
        
        with tf.GradientTape() as tape:
            output = self((img1, img2), training=True)
            loss = self.loss_function(label, output)
            
        gradients = tape.gradient(loss, self.trainable_variables)
        
        # update the state of the metrics according to loss
        # return a dictionary with metric names as keys and metric results as values





    # 6. test_step method
    def ...
        # same as train step (without parameter updates)






## Create a summary writer to log data

- use tf.summary.create_file_writer(log_path)

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/"

val_log_path = f"logs/"

train_summary_writer = ...

val_summary_writer = ...

## Write a training loop function

Arguments: 
 - the model to train, 
 - the data to train on, 
 - the data to test on, 
 - how many epochs to train, 
 - the train summary writer object to use for logging
 - the validation summary writer object to use for logging
 - a path to save trained model weights to

In [ ]:
def training_loop(...):

    # 1. iterate over epochs


        # 2. train steps on all batches in the training data



        # 3. log and print training metrics

        with train_summary_writer.as_default():
            ...



        # 4. reset metric objects



        # 5. evaluate on validation data



        # 6. log validation metrics

        with val_summary_writer.as_default():
            ...

        # 7. reset metric objects


    # 8. save model weights


## Use the training loop function to train the model

In [ ]:
# open the tensorboard logs
%tensorboard --logdir logs/

In [ ]:
# 1. instantiate model

# 2. choose a path to save the weights

save_path = ...

# 2. pass arguments to training loop function

training_loop(...)